# --------------------- Imperial Valley Birds data - Guy McCaskie ----------------------

In this section, I will take the cleaned data and combine it in a single file

In [74]:
# jupyter nbextension enable jupyter_tabnine --py
# importing all the dependencies 
import pandas as pd
import numpy as np

import os
import glob
import shlex

# docx file reading tools
import docx2txt
import textract
import sys

# import webscrapping tools
from urllib.request import urlretrieve
from bs4 import BeautifulSoup as bs

# import regular expression tools
import re
import string

# import date and time processing libraries
import datefinder
import datetime

import pickle
from statistics import mean

Next step is creating the appropiate functions for data extraction and cleaning from the text files. With the help of regular expressions, take out the useful text such as bird name and quantitiy according to the file formatting.

Most of the files are following a vague resembence - Hence, make it more readable for the regex script.


In [6]:
# antiword changed the file format from doc to docx - but still we have some leftover encoded data which antiword 
# was unable to read


def data_clean(text):
    
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('\\\\[\w\d]{3}', "", text)
    text = re.sub('\sand\s|,and\s', '', text)
    text = re.sub('BrownPelican', 'Brown Pelican', text)
    text = re.sub('Gray Flycatcher', 'Gray Flycatcher (1)', text)
    text = re.sub('Western Wood Pewee', 'Western Wood Pewee (1)', text)
    text = re.sub('Yellow-headed Blackbird Western Meadowlark', 'Western Meadowlark', text )
     
 
    return text

These text files can later be used to extract route map for the survey. In the first version, I only used Bird names and the quantities they were seen.

In [7]:
# remove_exra_text method will get rid of most of the text other than birds_name

def remove_extra_text(text):
    reg = re.compile('[\.Ss\s][p\s]ecies seen')
    spn = reg.search(text)
    
    # A more stable/consistent regex is to look for the first two unique words and use the wild card for the next 
    # spn.span()[1]+14 matches and ignore the text including 'Species seen and or heard '
    
    if spn != None:
        return text[spn.span()[1]+14:]
    else:
        return text
    

In [8]:
# Wriiting a regex for three or less worded bird name that might include hiphen '-' in it
# for quantity, I am looking for the first numeric entry in tne bracket

def extract_bird_data(text):
    return dict(re.findall('([A-Z][a-z’-]+\s[A-Z][A-Za-z’-]+\s[A-Z][A-Za-z’-]+\s[A-Z][A-Za-z’-]+|[A-Z][a-z’-]+\s[A-Z][A-Za-z’-]+\s[A-Z][A-Za-z’-]+|[A-Z][a-z’-]+\s[A-Z][A-Za-z’-]+|[A-Z][A-Za-z’-]+)\s*?\((\d+)', text))

In [9]:
def reformat_names(text):
    return re.sub('- ', '-', text)

## Applying the Data extraction and cleaning methods to all the files

Storing a set of the extracted dataframes as csv files for backup


In [102]:
# Apply the methods and store the results in a list df_list that can later be used for data concatenation

df_list = []
directory = "/home/jawwad/Data - guy's surveys/Guy_s surveys"
# try:
for file in os.listdir(directory):

    path = os.path.join(directory, file)

    with open(path, 'rb') as f:
        open_file = str(f.read())
    print(file)
    
    open_file = data_clean(open_file)
    match = (re.search('(\d+)\sto\s(\d+)', open_file))
    if match is not None:
        temp = str(match.group())
    l = re.findall('\d+', temp)
    l = list(map(int , l))
    open_file = remove_extra_text(open_file)
    open_file = reformat_names(open_file)
    open_file = extract_bird_data(str(open_file))
    open_file.update({'temp_low' : l[0], 'temp_high' : l[1], 'temp_mean' : mean(l)})
    
    # removing the file extension from the filename and using the string as index name for date of survey
    file = re.sub('.docx', '', file)
    df = pd.DataFrame(open_file, index =[file])
    
    
#    dest_path = os.path.splitext(path)[0] + ".xlsx"
#     df.to_excel(dest_path, index = False)

    df_list.append(df)
# except:
#     pass
        

2018-09-12.docx
2016-05-28.docx
2019-11-22.docx
2018-10-23.docx
2016-08-27.docx
2019-07-19.docx
2016-08-11.docx
2017-12-28.docx
2019-08-14.docx
2016-09-10.docx
2018-07-18.docx
2017-04-26.docx
2017-03-15.docx
2018-02-13.docx
2017-01-04.docx
2016-11-12.docx
2019-03-06.docx
.~lock.2016-10-13.xlsx
2019-08-26.docx
2018-12-12.docx
2016-08-05.docx
2019-12-17.docx
2016-08-23.docx
2017-03-21.docx
2018-01-31.docx
2018-06-24.docx
2018-03-17.docx
2019-05-31.docx
2018-07-04.docx
2017-10-18.docx
2018-06-20.docx
2018-07-28.docx
2016-09-23.docx
2016-02-28.docx
2018-11-28.docx
2020-01-31.docx
2018-01-14.docx
2019-06-14.docx
2018-03-08.docx
2017-09-20
2017-09-03
2019-09-25.docx
2017-03-28.docx
2018-11-07.docx
2016-11-05.docx
2017-05-08.docx
2020-02-25.docx
2016-11-29.docx
2018-09-04.docx
2018-09-19.docx
2020-02-21.docx
2018-03-03.docx
2018-05-23.docx
2019-03-22.docx
2019-10-02.docx
2016-07-07.docx
2020-01-25.docx
2019-10-11.docx
2017-10-11
2017-04-02.docx
2016-10-20.docx
2017-04-19.docx
2019-04-05.docx


Next step involves Concatenating the data from the list of dataframes 


In [103]:
# indexing is based on datatime format date
complete_birds_df = pd.concat(df_list)

To make the data easier to interpret take a peek and remove any errors. 

In [104]:
complete_birds_df.describe()

,temp_low,temp_high,temp_mean
count,136.000000,136.000000,136.000000
mean,61.419118,90.191176,75.805147
std,14.208790,14.570653,14.083635
min,35.000000,60.000000,47.500000
25%,50.000000,78.750000,64.375000
50%,60.000000,90.000000,75.000000
75%,75.000000,105.000000,87.500000
max,85.000000,115.000000,100.000000


In [106]:
# This step involves changing the data type from object to integer - to save space and use it fo analysis
# replacing missing values (NaN) with 0 - It gives literal meaning and easier for analysis 

pd.options.display.max_seq_items = 4000
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

complete_birds_df = complete_birds_df.apply(pd.to_numeric).fillna(0).astype(int)

complete_birds_df = complete_birds_df.drop('.~lock.2016-10-13.xlsx')

complete_birds_df.index = pd.to_datetime(complete_birds_df.index)

complete_birds_df

,Blue-winged Teal,Cinnamon Teal,Northern Shoveler,Gadwall,Mallard,Northern Pintail,Green-winged Teal,Ruddy Duck,Gambels Quail,Pied-billed Grebe,Eared Grebe,Western Grebe,Clarks Grebe,Rock Pigeon,Eurasian Collared-Dove,Inca Dove,Common Ground-Dove,White-winged Dove,Mourning Dove,Greater Roadrunner,Lesser Nighthawk,Black-chinned Hummingbird,Annas Hummingbird,Costas Hummingbird,Ridgways Rail,Virginia Rail,Sora,Common Gallinule,American Coot,Sandhill Crane,Black-necked Stilt,American Avocet,Black-bellied Plover,Snowy Plover,Semipalmated Plover,Killdeer,Long-billed Curlew,Marbled Godwit,Ruddy Turnstone,Stilt Sandpiper,Bairds Sandpiper,Least Sandpiper,Pectoral Sandpiper,Western Sandpiper,Western Sandpipers,Long-billed Dowitcher,Wilsons Snipe,Spotted Sandpiper,Lesser Yellowlegs,Willet,Greater Yellowlegs,Wilsons Phalarope,Red-necked Phalarope,Parasitic Jaeger,Laughing Gull,Ring-billed Gull,Western Gull,Yellow-footed Gull,California Gull,Herring Gull,Caspian Tern,Black Tern,Forsters Tern,Black Skimmer,Neotropic Cormorant,Double-crested Cormorant,American White Pelican,Brown Pelican,Least Bittern,Great Blue Heron,Great Egret,Snowy Egret,Cattle Egret,Green Heron,Black-crowned Night-Heron,White-faced Ibis,Turkey Vulture,Osprey,White-tailed Kite,Northern Harrier,Coopers Hawk,Red-tailed Hawk,Great Horned Owl,Burrowing Owl,Gila Woodpecker,Ladder-backed Woodpecker,American Kestrel,Western Wood-Pewee,Willow Flycatcher,Western Flycatcher,Black Phoebe,Vermilion Flycatcher,Western Kingbird,Loggerhead Shrike,Warbling Vireo,Common Raven,Purple Martin,Tree Swallow,Northern Rough-winged Swallow,Bank Swallow,Cliff Swallow,Barn Swallow,Verdin,House Wren,Marsh Wren,Blue-gray Gnatcatcher,Black-tailed Gnatcatcher,Northern Mockingbird,European Starling,House Sparrow,House Finch,Lesser Goldfinch,Aberts Towhee,Chipping Sparrow,Savannah Sparrow,Song Sparrow,Yellow-headed Blackbird,Western Meadowlark,Bullocks Oriole,Red-winged Blackbird,Brown-headed Cowbird,Brewers Blackbird,Great-tailed Grackle,Orange-crowned Warbler,Nashville Warbler,Common Yellowthroat,Yellow Warbler,Black-throated Gray Warbler,Wilsons Warbler,Western Tanager,Black-headed Grosbeak,temp_low,temp_high,temp_mean,American Wigeon,Redhead,Red-shouldered Hawk,Gull-billed Tern,Roadrunner,Says Phoebe,Ash-throated Flycatcher,Horned Lark,Phainopepla,Blue Grosbeak,Bronzed Cowbird,Greater White-fronted Goose,Snow Goose,Rosss Goose,Canvasback,Ring-necked Duck,Greater Scaup,Lesser Scaup,Bufflehead,Hooded Merganser,Red-breasted Merganser,Sharp-shinned Hawk,Ferruginous Hawk,Belted Kingfisher,Northern Flicker,Merlin,Peregrine Falcon,Rock Wren,Ruby-crowned Kinglet,American Pipit,White-crowned Sparrow,Lincolns Sparrow,Yellow-rumped Warbler,Ruff,Dunlin,Bonapartes Gull,Franklins Gull,Red-naped Sapsucker,American Robin,Cedar Waxwing,Dark-eyed Junco,American Redstart,Solitary Sandpiper,Rufous Hummingbird,Pacific-slope Flycatcher,Cassins Vireo,Lucys Warbler,Gillivrays Warbler,Hermit Warbler,Yellow-breasted Chat,Black-throated Sparrow,Lazuli Bunting,Whimbrel,Short-billed Dowitcher,Reddish Egret,Canada Goose,Flicker,Bells Vireo,Western Bluebird,Golden-crowned Sparrow,Hooded Oriole,American Bittern,Common Tern,Cactus Wren,Townsends Warbler,Red Knot,Lesser Black-backed Gull,Vauxs Swift,Calliope Hummingbird,Sanderling,Common Loon,Barn Owl,Hammonds Flycatcher,Swainsons Thrush,Brown Thrasher,Summer Tanager,Brant,Surf Scoter,Common Goldeneye,Horned Grebe,Heermanns Gull,Rough-winged Swallow,Lark Sparrow,Common Merganser,Mountain Plover,Prairie Falcon,Harriss Sparrow,Cackling Goose,Ring-necked Pheasant,Hermit Thrush,Tundra Swan,Semipalmated Sandpiper,Mew Gull,Mountain Bluebird,Vesper Sparrow,Bells Sparrow,Glaucous-winged Gull,Violet-green Swallow,Sage Thrasher,Yellow-billed Cuckoo,Pine Siskin,Niland,Olive-sided Flycatcher,Greater Lesser Scaup,Thayers Gull,Bald Eagle,Brewers Sparrow,Little Blue Heron,Sabines Gull,Red-breasted Nuthatch,Great Burrowing Owl,American Wigeon Mallard,American Golden-Plover,Roseate Spoonbill,May,Rid

In [107]:
complete_birds_df.gt(0).sum().sort_values()

Gray Flycatcher                    1
Niland                             1
Little Blue Heron                  1
Bells Sparrow                      1
American Wigeon Mallard            1
Western Wood Pewee                 1
May                                1
Sharp-tailed Sandpiper             1
Ovenbird                           1
Mockingbird                        1
Thayers Gull                       1
Lark Bunting                       1
Glossy Ibis                        1
Northern Parula                    1
Huttons Vireo                      1
Barrows Goldeneye                  1
Greater Lesser Scaup               1
Swamp Sparrow                      1
Yellow-throated Vireo              1
Pacific Loon                       1
Black-and-white Warbler            1
Eastern Kingbird                   1
Glaucous Gull                      1
Townsends Solitaire                1
Red-breasted Sapsucker             1
Allens Hummingbird                 1
Blackpoll Warbler                  1
B

In [108]:
 stop_list = ['May','Niland', 'Gull']
#              'The black primaries','Iceland', 'The single']
# stop_list_1 = ['The  single', 'All four', , 'Obsidian Butte to the west end of Young Road',
#                'Yellow-headed Blackbird Western Meadowlark']

# stop_list_2 = ['end of Young Road','Unit in the morning','were pale reddish',
#                'as it fed','mid-morning with about fifteen','I noticed a single']

complete_birds_df = complete_birds_df.drop(stop_list, axis = 1)

In [109]:
col = complete_birds_df.columns
pd.Series(col).sort_values()

112                    Aberts Towhee
280               Allens Hummingbird
31                   American Avocet
195                 American Bittern
28                     American Coot
245           American Golden-Plover
86                  American Kestrel
163                   American Pipit
175                American Redstart
172                   American Robin
66            American White Pelican
134                  American Wigeon
244          American Wigeon Mallard
22                 Annas Hummingbird
140          Ash-throated Flycatcher
40                  Bairds Sandpiper
238                       Bald Eagle
99                      Bank Swallow
205                         Barn Owl
101                     Barn Swallow
265                Barrows Goldeneye
229                    Bells Sparrow
191                      Bells Vireo
157                Belted Kingfisher
273                     Bewicks Wren
90                      Black Phoebe
276                     Black Scoter
6

In [110]:
complete_birds_df.shape

(135, 283)

In [111]:
complete_birds_df.index = pd.to_datetime(complete_birds_df.index)

In [112]:
# Storing the dataframe as copy for excel analysis

dest_path = os.path.splitext(path)[0] + ".xlsx"
complete_birds_df.to_excel(dest_path, index = False)

In [113]:
# So far, everything looks good - Let's pickle the DataFrame

with open('birds_pickle.pkl', 'wb') as pickle_file:
    pickle.dump(complete_birds_df, pickle_file)

In [92]:
# with open('birds_pickle.pkl', 'rb') as pickle_file:
#     pickle_load = pickle.load(pickle_file)